In [ ]:
# default imports
import numpy as np

# custom imports
from NanTex_backend.data_preparation import img_augmentation as img_augmentation

In [ ]:
## Setup image augmentation
train_augmentation_pipeline = img_augmentation.train_augmentation_pipeline
val_augmentation_pipeline = img_augmentation.val_augmentation_pipeline

## Rearrange original WU overlays and save them

In [3]:
# default imports
import os
from tqdm.notebook import tqdm
from ezRay import MultiCoreExecutionTool

# custom
from NanTex_backend.Util.pyDialogue import pyDialogue as pyD

In [15]:
# grab files
files = {os.path.basename(f)[:-4] + "_rearranged_aug25.npy": f for f in pyD.askFILES()}

In [ ]:
# configure ezRay
instance_metadata: dict = {
    "num_cpus": 12,  # number of cpus to use
    "num_gpus": 0,  # number of gpus to use
    "address": None,  # remote cluster address. None for local.
}

# setup ezRay
MultiCore = MultiCoreExecutionTool(instance_metadata=instance_metadata)

# launch ray dashboard (optional)
MultiCore.launch_dashboard()


# define any task
def do_something(file, path, output_dir) -> int:
    data = np.load(path)

    # rearrange the data
    rearranged_data = np.stack([data[1, ...], data[2, ...], data[3, ...], data[0, ...]])

    # save the rearranged data
    save_path = os.path.join(output_dir, file)
    np.save(save_path, rearranged_data)
    return 0

2025-08-21 17:10:16,127	INFO worker.py:1918 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


No Runtime Data provided. Use the "update_data()" method to update the Runtime Data prior to running methods.


Scheduling Workers:   0%|          | 0/3072 [00:00<?, ?it/s]

Workers:   0%|          | 0/3072 [00:00<?, ?it/s]

CPU usage:   0%|          |

RAM usage:   0%|          |

In [13]:
MultiCore.RuntimeArchive = {}

In [16]:
# prepare your data in a dictionary. They keys work as identifiers, while the values should be dictionaries matching the function signature.
data = {
    i: {"file": file, "path": path, "output_dir": output_dir}
    for i, (file, path, output_dir) in enumerate(
        zip(
            list(files.keys()),
            list(files.values()),
            ["P:\\WÜRZBURG_2025_DATA\\test_data_aug25"] * len(files),
        )
    )
}

# pass the data to ezRay
MultiCore.update_data(data)

# run the task
MultiCore.run(do_something)

# get the results
results_first_task = MultiCore.get_results()

Previous results detected and moved to RuntimArchive.


Scheduling Workers:   0%|          | 0/480 [00:00<?, ?it/s]

Workers:   0%|          | 0/480 [00:00<?, ?it/s]

CPU usage:   0%|          |

RAM usage:   0%|          |

In [14]:
output_dir = pyD.askDIR()

for file in tqdm(files):
    # load the file
    data = np.load(files[file])

    # rearrange the data
    rearranged_data = np.stack([data[1, ...], data[2, ...], data[3, ...], data[0, ...]])

    # save the rearranged data
    save_path = os.path.join(output_dir, file)
    np.save(save_path, rearranged_data)

  0%|          | 0/3072 [00:00<?, ?it/s]

KeyboardInterrupt: 